In [1]:
from lsst.ts.standardscripts.auxtel.integration_tests.at_tracking import ATTracking

In [39]:
import logging
import yaml
from lsst.ts import salobj
import SALPY_ATPtg
import asyncio

In [3]:
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle



In [4]:
script = ATTracking(index=11)

In [5]:
script.log.setLevel(logging.INFO)
script.log.addHandler(logging.StreamHandler())

In [29]:
print(salobj.State(script.atptg.evt_summaryState.get().summaryState))
print(salobj.State(script.ataos.evt_summaryState.get().summaryState))
print(salobj.State(script.athexapod.evt_summaryState.get().summaryState))
print(salobj.State(script.atpneumatics.evt_summaryState.get().summaryState))
print(salobj.State(script.atmcs.evt_summaryState.get().summaryState))

State.ENABLED
State.ENABLED
State.FAULT
State.ENABLED
State.ENABLED
ERROR! Session/line number was not unique in database. History logging moved to new session 50


In [30]:
await salobj.set_summary_state(script.atptg, salobj.State.STANDBY, 
                               timeout=300)

await salobj.set_summary_state(script.ataos, salobj.State.STANDBY,settingsToApply="test", 
                               timeout=300)
await salobj.set_summary_state(script.athexapod, salobj.State.STANDBY, 
                             settingsToApply="Default1", timeout=300)
await salobj.set_summary_state(script.atpneumatics, salobj.State.STANDBY, timeout=300)
await salobj.set_summary_state(script.atmcs, salobj.State.STANDBY, timeout=300)

In [31]:
print(salobj.State(script.atptg.evt_summaryState.get().summaryState))
print(salobj.State(script.ataos.evt_summaryState.get().summaryState))
print(salobj.State(script.athexapod.evt_summaryState.get().summaryState))
print(salobj.State(script.atpneumatics.evt_summaryState.get().summaryState))
print(salobj.State(script.atmcs.evt_summaryState.get().summaryState))

State.STANDBY
State.STANDBY
State.STANDBY
State.STANDBY
State.STANDBY


In [32]:
await salobj.set_summary_state(script.atptg, salobj.State.ENABLED, timeout=300)
print(salobj.State(script.atptg.evt_summaryState.get().summaryState))
await salobj.set_summary_state(script.ataos, salobj.State.ENABLED,
                               settingsToApply="test", timeout=300)
print(salobj.State(script.ataos.evt_summaryState.get().summaryState))
await salobj.set_summary_state(script.athexapod, salobj.State.ENABLED, 
                               settingsToApply="Default1", timeout=300)
print(salobj.State(script.athexapod.evt_summaryState.get().summaryState))
await salobj.set_summary_state(script.atpneumatics, salobj.State.ENABLED, 
                               timeout=300)
print(salobj.State(script.atpneumatics.evt_summaryState.get().summaryState))
await salobj.set_summary_state(script.atmcs, salobj.State.ENABLED, 
                               timeout=300)
print(salobj.State(script.atmcs.evt_summaryState.get().summaryState))

State.ENABLED
State.ENABLED
State.ENABLED
State.ENABLED
State.ENABLED


In [33]:
time_data = await script.atptg.tel_timeAndDate.next(flush=False, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_atptg - Time.now()
script.log.info(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=45.*u.deg, az=45.*u.deg, obstime=curr_time_atptg.tai, location=script.location)
cmd_radec = cmd_elaz.transform_to(ICRS)

# Start tracking
script.atptg.cmd_raDecTarget.set(
    targetName="atptg_atmcs_integration",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_icrs,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=0,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_target,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)
script.log.info(f"raDecTarget ra={script.atptg.cmd_raDecTarget.data.ra!r} hour; "
              f"declination={script.atptg.cmd_raDecTarget.data.declination!r} deg")
script.atmcs.evt_target.flush()
script.atmcs.evt_allAxesInPosition.flush()
# script.ataos.cmd_enableCorrection.set(hexapod=True)
# await script.ataos.cmd_enableCorrection.start(timeout=10)
ack_id = await script.atptg.cmd_raDecTarget.start(timeout=2)
script.log.info(f"raDecTarget command result: {ack_id.ack.result}")



Time error=-200.11 sec
raDecTarget ra=5.845275969041962 hour; declination=4.343515466114815 deg
raDecTarget command result: Done : OK


In [34]:
while True:
    in_position = await script.atmcs.evt_allAxesInPosition.next(flush=False, timeout=20)
    script.log.info(f"Got {in_position.inPosition}")
    if in_position.inPosition:
        break

Got 1


In [11]:
await script.atptg.cmd_stopTracking.start()

CommandIdAck(cmd_id=1558972333, ack=(ack=303, error=0, result='Done : OK'))

In [35]:
corr = script.ataos.evt_correctionEnabled.get()

In [36]:
corr.hexapod

0

In [14]:
script.ataos.cmd_disableCorrection.set(disableAll=True)
await script.ataos.cmd_disableCorrection.start()

CommandIdAck(cmd_id=1558972333, ack=(ack=303, error=0, result='Done'))

In [37]:
await script.ataos.cmd_applyCorrection.start()

--> 0.0 0.0 6.43098343929 -0.0 0.0 -0.0


CommandIdAck(cmd_id=1558972338, ack=(ack=303, error=0, result='Done'))

In [41]:
for i in range(10):
    coro = script.athexapod.evt_positionUpdate.next(flush=True, timeout=5)
    print("Apply correction...")
    await script.ataos.cmd_applyCorrection.start()
    id_data = await coro
    print('-->', 
          id_data.positionX, 
          id_data.positionY, 
          id_data.positionZ, 
          id_data.positionU, 
          id_data.positionV, 
          id_data.positionW)
    print("waiting 15s...")
    await asyncio.sleep(15.)



Apply correction...
--> 0.0 0.0 6.48267307205 -0.0 0.0 -0.0
waiting 30s...
Apply correction...
--> 0.0 0.0 6.48441167056 -0.0 0.0 -0.0
waiting 30s...
Apply correction...
--> 0.0 0.0 6.48612811411 -0.0 0.0 -0.0
waiting 30s...
Apply correction...
--> 0.0 0.0 6.48783592978 -0.0 0.0 -0.0
waiting 30s...
Apply correction...
--> 0.0 0.0 6.48945347399 -0.0 0.0 -0.0
waiting 30s...
Apply correction...
--> 0.0 0.0 6.491141798 -0.0 0.0 -0.0
waiting 30s...
Apply correction...
--> 0.0 0.0 6.49282388292 -0.0 0.0 -0.0
waiting 30s...
Apply correction...
--> 0.0 0.0 6.49449721546 -0.0 0.0 -0.0
waiting 30s...
Apply correction...
--> 0.0 0.0 6.49617985477 -0.0 0.0 -0.0
waiting 30s...
Apply correction...
--> 0.0 0.0 6.49783309647 -0.0 0.0 -0.0
waiting 30s...
